# DSI Summer Workshops Series


Peggy Lindner<br>
Center for Advanced Computing & Data Science (CACDS)<br>
Data Science Institute (DSI)<br>
University of Houston  
plindner@uh.edu 


This Notebook is available at:
http://bitly.com/UHDSInotebook1

You can already download it! Use the "Save Link as" method.

Please make sure you have a copy of R up and running, as well as a Python 3 installation (ideally from Anacodna).

## Goals for today

Understand basics of text analysis using R

(well enough so that you can Google your problems, find the answer, and implement it.)

#### More specifically

1. Up and running with R & IPython
2. Understand a basic exploratory data analysis workflow
3. Basics of R and Topic Modeling 

### The Exploratory Analysis Workflow



![](https://raw.github.com/peggylind/DSI_Summer_Workshops/master/images/data-science-workflow.png)
Image source: Hadley Wickham, R for Data Science

#### Why R and not Python 

It's good for data exploration! 

## Getting yourself ready

### First: Install software on your computer

* R [CRAN](https://www.anaconda.com/download/)
* Python[Anaconda](https://www.anaconda.com/download/)




### Second: Prep your R environment
On a Mac open a terminal and start R

![](https://raw.github.com/peggylind/DSI_Summer_Workshops/master/images/on-a-mac.png)

On Windows: Open the Anaconda Command line and start R

![](https://raw.github.com/peggylind/DSI_Summer_Workshops/master/images/anaconda-start.png)

![](https://raw.github.com/peggylind/DSI_Summer_Workshops/master/images/windows.png)

Now let's install some packages ...

```
> install.packages(c('repr', 'IRdisplay', 'evaluate', 'crayon', 'pbdZMQ', 'devtools', 'uuid', 'digest'))
> devtools::install_github('IRkernel/IRkernel')
```

When you see "Please select a CRAN mirror" , well select one.

![](https://raw.github.com/peggylind/DSI_Summer_Workshops/master/images/cran-repo.png)

... one last step - installing the Kernel

```
> IRkernel::installspec()
```

Now we can close the R environment (but leave your terminal and console open)


```
> quit()
```

Say "N" (no) when asked to save the workspace.

### Jupyter Notebooks is what we will be going to use

We are now ready to start up our Jupyter Environment from the terminal or the console:

```
$ jupyter notebook --notebook-dir C:/Users/[your username]

or on a Mac

$ jupyter notebook --notebook-dir /Users/[your username]
```

And your browser should open at the address: http://localhost:8888/tree

![](https://raw.github.com/peggylind/DSI_Summer_Workshops/master/images/jupyter.png)

#### Open the downloaded notebook on your computer

![](https://raw.github.com/peggylind/DSI_Summer_Workshops/master/images/second-screen.png)

https://raw.github.com/peggylind/DSI_Summer_Workshops/master/DSI%20Workshop%20Summer%202018a.ipynb

Cells can be Markdown (like this one) or code

![](https://raw.github.com/peggylind/DSI_Summer_Workshops/master/images/thrid-screen.png)

#### To start off with

Make sure you hit `Shift-Enter` or `Ctrl-Enter` when you are done.

In [3]:
2 + 2

4

#### Frequently used R Packages

Data aquisition

* [requests](http://docs.python-requests.org/en/latest/) Web scraping
* [mechanize](http://wwwsearch.sourceforge.net/mechanize/) Web scraping
* [BeautifulSoup](http://www.crummy.com/software/BeautifulSoup/) HTML cleaning
* [Tweepy](http://tweepy.github.io) Access to Twitter API
* [Marvel Comics](http://developer.marvel.com) Access to Marvel Universe
* [database](https://dataset.readthedocs.org/en/latest/) Relational database/SQL



Analysis

* [NLTK](http://nltk.org) Natural Language Toolkit. There's a [book](http://nltk.org/book/)
* [scikit-learn](http://scikit-learn.org/stable/) Machine learning
* [pandas](http://pandas.pydata.org) Data management and analysis. There is a book.
* [gensim](http://radimrehurek.com/gensim/) Topic modeling
* [NetworkX](http://networkx.github.io) Network analysis
* [matplotlib](http://matplotlib.org) Plotting* [NumPy](http://www.numpy.org) and [scipy](http://scipy.org) Computational backbone
* [rpy](http://rpy.sourceforge.net) Python bindings for R
